In [5]:
import mne
import numpy as np
from scipy.io import loadmat, savemat

def parseBci42aFile(dataPath, labelPath, epochWindow = [0,4], chans = list(range(22))):
    '''
    Parse the bci42a data file and return an epoched data. 

    Parameters
    ----------
    dataPath : str
        path to the gdf file.
    labelPath : str
        path to the labels mat file.
    epochWindow : list, optional
        time segment to extract in seconds. The default is [0,4].
    chans  : list : channels to select from the data. 
    
    Returns
    -------
    data : an EEG structure with following fields:
        x: 3d np array with epoched EEG data : chan x time x trials
        y: 1d np array containing trial labels starting from 0
        s: float, sampling frequency
        c: list of channels - can be list of ints. 
    '''
    eventCode = ['768'] # start of the trial at t=0
    fs = 250
    offset = 2
    
    #load the gdf file using MNE
    raw_gdf = mne.io.read_raw_gdf(dataPath, stim_channel="auto")
    raw_gdf.load_data()
    print(raw_gdf)
    gdf_event_labels = mne.events_from_annotations(raw_gdf)[1]
    print(gdf_event_labels)
    eventCode = [gdf_event_labels[x] for x in eventCode]

    gdf_events = mne.events_from_annotations(raw_gdf)[0][:,[0,2]].tolist()
    eeg = raw_gdf.get_data()
    
    # drop channels
    if chans is not None:
        eeg = eeg[chans,:]
    
    print(eeg.shape)

    # #Epoch the data
    # events = [event for event in gdf_events if event[1] in eventCode]
    # y = np.array([i[1] for i in events])
    # epochInterval = np.array(range(epochWindow[0]*fs, epochWindow[1]*fs))+offset*fs
    # print(events)
    # print(len(events))
    # x = np.stack([eeg[:, epochInterval+event[0] ] for event in events], axis = 2)
    
    # print(x.shape)
    
    # # Multiply the data with 1e6
    # x = x*1e6
    
    # # have a check to ensure that all the 288 EEG trials are extracted.
    # assert x.shape[-1] == 288, "Could not extracted all the 288 trials from GDF file: {}. Manually check what is the reason for this".format(dataPath)

    # #Load the labels
    # y = loadmat(labelPath)["classlabel"].squeeze()
    # # change the labels from [1-4] to [0-3] 
    # y = y -1
    
    # data = {'x': x, 'y': y, 'c': np.array(raw_gdf.info['ch_names'])[chans].tolist(), 's': fs}
    # print(data['x'].shape, data['y'].shape)


In [6]:
filename = '../data/bci42a/originalData/A01T.gdf'
labelpath = '../data/bci42a/originalData/A01T.mat'

data = parseBci42aFile(filename, labelpath)

Extracting EDF parameters from d:\codes\BCI-VR\FBCNet\data\bci42a\originalData\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


<d:\anaconda3\envs\FBCNet\lib\site-packages\decorator.py:decorator-gen-157>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.


<RawGDF | A01T.gdf, 25 x 672528 (2690.1 s), ~128.3 MB, data loaded>
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
{'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
(22, 672528)


In [20]:
import torch.nn as nn

class VarLayer(nn.Module):
    '''
    The variance layer: calculates the variance of the data along given 'dim'
    '''
    def __init__(self, dim):
        super(VarLayer, self).__init__()
        self.dim = dim

    def forward(self, x):
        return x.var(dim = self.dim, keepdim= True)

In [25]:
import torch

x = torch.Tensor([1, 1, 1])
y = torch.Tensor([1, 2, 3])

var = VarLayer(0)
print(var(x), var(y))

tensor([0.]) tensor([1.])
